In [1]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

In [5]:
mol_name = 'H2O'
# mol_name = 'H2'

# mol_name = 'LiH'

In [6]:
xyz_file = xyz_from_pubchem(mol_name)
print(xyz_file)

Unable to find structure info in the PubChem databasefor the specified molecule LiH.
2
 
Li	3	0	0
H	2	0	0



In [11]:
from symred.chem import Draw_molecule

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="stick")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [12]:
xyz_file = '4\n \nH\t0\t0\t0\nH\t1\t0\t0\nH\t2\t0\t0\nH\t3\t0\t0'

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="sphere")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
from symred.chem import PySCFDriver

In [ ]:
basis = 'STO-3G'
convergence = 1e6
charge=0
max_hf_cycles=50
ram = 8_000
run_mp2  = True,
run_cisd = True,
run_ccsd = True,
run_fci  = True

In [ ]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [ ]:
pyscf_obj.run_pyscf()

In [ ]:
2*pyscf_obj.pyscf_hf.mol.nao

In [ ]:
from symred.chem import FermionicHamilt

In [ ]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_operator()

H_ferm.fermionic_molecular_hamiltonian

In [ ]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


In [ ]:
hf_ket = H_ferm.hf_ket
hf_ket.shape

In [ ]:
## check
from functools import reduce
state = {0: np.array([[1],[0]]), 1: np.array([[0],[1]])}
ll = [state[bit] for bit in hf_state]
kket = reduce(np.kron, ll)
all(kket[:,0] == hf_ket)

In [ ]:
print(H_ferm.scf_method.energy_tot())
print(pyscf_obj.pyscf_hf.energy_tot())

H_mat = H_ferm.get_sparse_ham()
hf_ket.conj().T @ H_mat @ hf_ket

In [ ]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
ei = fij[:nocc]
ai = fij[nocc:]
abgij = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (
        ei.reshape(1, 1, -1, 1) + ei.reshape(1, 1, 1, -1) - ai.reshape(-1, 1, 1, 1) - ai.reshape(1, -1, 1, 1))
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

In [ ]:
pyscf_obj.pyscf_mp2.e_corr

In [ ]:
## key

# i,j,I,J are occupied molecular orbitals
# a,b,A,B are virtual molecular orbitals

# lower case = spin down !!!
# upper case = spin up   !!!

In [ ]:
tIjAb=np.einsum('abij -> ijab', amplitudes, optimize='greedy')

tIjAb.shape

In [ ]:
tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')
tijab.shape

In [ ]:
tIjAb=tIjAb
tiJaB=tIjAb
tijab=tijab
tIJAB=tijab

In [ ]:
def alpha_occupied(i):
    return 2 * i

def alpha_unoccupied(i, n_alpha_electrons):
    return 2 * (i + n_alpha_electrons)

def beta_occupied(i):
    return 2 * i + 1

def beta_unoccupied(i, n_beta_electrons):
    return 2 * (i + n_beta_electrons) + 1

In [ ]:
# restricted=True

# threshold = 1e-12
# excite_op =FermionOperator()
# nvirt = tIjAb.shape[2]
# nocc = tIjAb.shape[0]
# assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)


# for (I, J, A, B), value in np.ndenumerate(tIJAB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         I_occ = alpha_occupied(I)
#         J_occ = alpha_occupied(J)
#         A_vir = alpha_unoccupied(A, nocc)
#         B_vir = alpha_unoccupied(B, nocc)
#         excite_op+= FermionOperator(f'{A_vir}^ {I_occ} {B_vir}^ {J_occ}', value/2)
        
#         if (restricted):
#             # spin down
#             I_occ_d = beta_occupied(I)
#             J_occ_d = beta_occupied(J)
#             A_vir_d = beta_unoccupied(A, nocc)
#             B_vir_d = beta_unoccupied(B, nocc)
#             excite_op+= FermionOperator(f'{A_vir_d}^ {I_occ_d} {B_vir_d}^ {J_occ_d}', value/2)

# for (i, j, a, b), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#             i_occ = beta_occupied(i)
#             j_occ = beta_occupied(j)
#             a_vir = beta_unoccupied(a, nocc)
#             b_vir = beta_unoccupied(b, nocc)
#             excite_op+= FermionOperator(f'{a_vir}^ {i_occ} {b_vir}^ {j_occ}', value/2)
            
            
# for (I, j, A, b), value in np.ndenumerate(tIjAb):
#     if not np.isclose(value, 0.0, atol=threshold):
#         I_occ = alpha_occupied(I)
#         j_occ_d = beta_occupied(j)
#         A_vir = alpha_unoccupied(A, nocc)
#         b_vir_d = beta_unoccupied(b, nocc)
        
#         excite_op+= FermionOperator(f'{A_vir}^ {I_occ} {b_vir_d}^ {j_occ_d}', value/2)
        
#         if (restricted):
#             ## note tIjAb == tiJaB
#             i_occ_d = beta_occupied(I)
#             J_occ = alpha_occupied(j)
#             a_vir_d = beta_unoccupied(A, nocc)
#             B_vir = alpha_unoccupied(b, nocc)
#             excite_op+= FermionOperator(f'{a_vir_d}^ {i_occ_d} {B_vir}^ {J_occ}', value/2)

        
#             # Add missing same--spin amplitudes in restricted / closed-shell cases:
#             # T_IJAB = T_ijab = T_IjAb - T_IjBa
#             same_spin_amp = tIjAb[I,j,A,b] / 2. - tiJaB[I,j,b,A] / 2.
            
#             # alpha
#             excite_op+= FermionOperator(f'{A_vir}^ {I_occ} {B_vir}^ {J_occ}', same_spin_amp / 2)
            
#             # beta
#             excite_op+= FermionOperator(f'{b_vir_d}^ {j_occ_d} {a_vir_d}^ {i_occ_d}', same_spin_amp / 2)


# ######


# print(len(list(excite_op)))
# excite_op

In [ ]:
# restricted=True

# threshold = 1e-12
# excite_op =FermionOperator()
# nvirt = tIjAb.shape[2]
# nocc = tIjAb.shape[0]
# assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)


# for (i,j,a,b), value in np.ndenumerate(tIJAB):
#     # up,up,up,up
#     if not np.isclose(value, 0.0, atol=threshold):
#         # alpha
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                                       value/2)
        
#         if (restricted):
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                           value/2)

# for (i, j, a, b), value in np.ndenumerate(tijab):
#     # down,down,down,down
#     if not np.isclose(value, 0.0, atol=threshold):
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                       value/2)

    
# for (i, j, a, b), value in np.ndenumerate(tIjAb):
#     # up,down,up,down
#     if not np.isclose(value, 0.0, atol=threshold):
        
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                       value/2)
        
#         if (restricted):
#             # down,up ,up,down
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                                           value/2)

        
#             # Add missing same--spin amplitudes in restricted / closed-shell cases:
#             # T_IJAB = T_ijab = T_IjAb - T_IjBa
#             same_spin_amp = tIjAb[i,j,a,b] / 2. - tiJaB[i,j,b,a] / 2.
            
#             # alpha
#             excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               same_spin_amp/2)
            
#             # beta
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                               same_spin_amp/2)

            
# for (i, j, a, b), value in np.ndenumerate(tIJAB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               value/2)
        
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               value/2)
        
# #         if (restricted):
# #             ## note tIjAb == tiJaB
# #             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
# #                                           value/2)


# ######


# print(len(list(excite_op)))
# excite_op

In [ ]:
# restricted=True

# threshold = 1e-12
# excite_op =FermionOperator()
# nvirt = tIjAb.shape[2]
# nocc = tIjAb.shape[0]
# assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)


# for (i,j,a,b), value in np.ndenumerate(tIJAB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         # up,up,up,up
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                                       value/2)
        
#         if (restricted):
#             # down,down,down,down
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                           value/2)

# for (i, j, a, b), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#         # down,down,down,down
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                       value/2)

    
# for (i, j, a, b), value in np.ndenumerate(tIjAb):
#     if not np.isclose(value, 0.0, atol=threshold):
#         # up, up, down, down
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                                       value/2)
        
#         if (restricted):
#             # down, down, up, up
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                                           value/2)

        
#             # Add missing same--spin amplitudes in restricted / closed-shell cases:
#             # T_IJAB = T_ijab = T_IjAb - T_IjBa
#             same_spin_amp = tIjAb[i,j,a,b] / 2. - tiJaB[i,j,b,a] / 2.
            
#             # up up, up, up
#             excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               same_spin_amp/2)
            
#             # down, down, down, down
#             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
#                               same_spin_amp/2)

            
# for (i, j, a, b), value in np.ndenumerate(tIJAB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         # up, down, up, up
#         excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               value/2)
        
#         excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
#                               value/2)
        
# #         if (restricted):
# #             ## note tIjAb == tiJaB
# #             excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
# #                                           value/2)


# ######


# print(len(list(excite_op)))
# excite_op

In [ ]:
restricted=True

threshold = 1e-12
excite_op =FermionOperator()
nvirt = tIjAb.shape[2]
nocc = tIjAb.shape[0]
assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)


for (i,j,a,b), value in np.ndenumerate(tIJAB):
    # up,up,up,up
    if not np.isclose(value, 0.0, atol=threshold):
        # up,up,up,up
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
                                      value/2)
        
        if (restricted):
            # down,down,down,down
            excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
                                          value/2)

for (i, j, a, b), value in np.ndenumerate(tijab):
    # down,down,down,down
    if not np.isclose(value, 0.0, atol=threshold):
        # up,up,up,up
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
                                      value/2)

    
for (i, j, a, b), value in np.ndenumerate(tIjAb):
    if not np.isclose(value, 0.0, atol=threshold):
        # up, up, down, down
        excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
                                      value/2)
        
        if (restricted):
            # down, down, up, up
            excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
                                          value/2)

        
            # Add missing same--spin amplitudes in restricted / closed-shell cases:
            # T_IJAB = T_ijab = T_IjAb - T_IjBa
            same_spin_amp = tIjAb[i,j,a,b] / 2. - tiJaB[i,j,b,a] / 2.
            
            # up up, up, up
            excite_op+= FermionOperator(f'{alpha_unoccupied(a, nocc)}^ {alpha_occupied(i)} {alpha_unoccupied(b, nocc)}^ {alpha_occupied(j)}',
                              same_spin_amp/2)
            
            # down, down, down, down
            excite_op+= FermionOperator(f'{beta_unoccupied(a, nocc)}^ {beta_occupied(i)} {beta_unoccupied(b, nocc)}^ {beta_occupied(j)}',
                              same_spin_amp/2)



######


print(len(list(excite_op)))
excite_op

In [ ]:
double_amplitudes.shape

In [ ]:
# Store doubles, include factor of 1/2 for convention
T2IJAB_Amps = tIJAB
T2ijab_Amps = tijab
T2IjAb_Amps = tIjAb

double_amplitudes = np.zeros((2*pyscf_obj.pyscf_hf.mol.nao, ) * 4)

for (i, j, a, b), value in np.ndenumerate(T2IJAB_Amps):
    double_amplitudes[alpha_unoccupied(a),
                      alpha_occupied(i),
                      alpha_unoccupied(b),
                      alpha_occupied(j)] = value / 2.
    if (restricted):
        double_amplitudes[beta_unoccupied(a),
                          beta_occupied(i),
                          beta_unoccupied(b),
                          beta_occupied(j)] = value / 2.

for (i, j, a, b), value in np.ndenumerate(T2ijab_Amps):
    double_amplitudes[beta_unoccupied(a),
                      beta_occupied(i),
                      beta_unoccupied(b),
                      beta_occupied(j)] = value / 2.

for (i, j, a, b), value in np.ndenumerate(T2IjAb_Amps):
    double_amplitudes[alpha_unoccupied(a),
                      alpha_occupied(i),
                      beta_unoccupied(b),
                      beta_occupied(j)] = value / 2.

    if (restricted):
        double_amplitudes[beta_unoccupied(a),
                          beta_occupied(i),
                          alpha_unoccupied(b),
                          alpha_occupied(j)] = value / 2.

        # Add missing same--spin amplitudes in restricted / closed-shell cases:
        # T_IJAB = T_ijab = T_IjAb - T_IjBa

        same_spin_amp = T2IjAb_Amps[i,j,a,b] / 2. - T2IjAb_Amps[i,j,b,a] / 2.

        double_amplitudes[alpha_unoccupied(a),
                          alpha_occupied(i),
                          alpha_unoccupied(b),
                          alpha_occupied(j)] = same_spin_amp / 2.

        double_amplitudes[beta_unoccupied(a),
                          beta_occupied(i),
                          beta_unoccupied(b),
                          beta_occupied(j)] = same_spin_amp / 2.


In [ ]:
print(len(list(generator)))
generator

In [ ]:
#    # Store doubles, include factor of 1/2 for convention
#     for entry in T2IJAB_Amps:
#         i, j, a, b, value = entry
#         double_amplitudes[alpha_unoccupied(a),
#                           alpha_occupied(i),
#                           alpha_unoccupied(b),
#                           alpha_occupied(j)] = value / 2.
#         if (restricted):
#             double_amplitudes[beta_unoccupied(a),
#                               beta_occupied(i),
#                               beta_unoccupied(b),
#                               beta_occupied(j)] = value / 2.

#     for entry in T2ijab_Amps:
#         i, j, a, b, value = entry
#         double_amplitudes[beta_unoccupied(a),
#                           beta_occupied(i),
#                           beta_unoccupied(b),
#                           beta_occupied(j)] = value / 2.

#     for entry in T2IjAb_Amps:
#         i, j, a, b, value = entry
#         double_amplitudes[alpha_unoccupied(a),
#                           alpha_occupied(i),
#                           beta_unoccupied(b),
#                           beta_occupied(j)] = value / 2.

#         if (restricted):
#             double_amplitudes[beta_unoccupied(a),
#                               beta_occupied(i),
#                               alpha_unoccupied(b),
#                               alpha_occupied(j)] = value / 2.

In [ ]:
# threshold = 1e-8
# variables = {}
# if tIjAb is not None:
#     nvirt = tIjAb.shape[2]
#     nocc = tIjAb.shape[0]
#     assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)

#     for (I, j, A, b), value in np.ndenumerate(tIjAb):
#         if not np.isclose(value, 0.0, atol=threshold):
#             variables[(2 * (nocc + A), 2 * I, 2 * (nocc + b) + 1, j + 1)] = value
#     for (i, J, a, B), value in np.ndenumerate(tiJaB):
#         if not np.isclose(value, 0.0, atol=threshold):
#             variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + B), J)] = value
#     for (i, j, a, b), value in np.ndenumerate(tijab):
#         if not np.isclose(value, 0.0, atol=threshold):
#             variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + b) + 1, j + 1)] = value
#     for (I, J, A, B), value in np.ndenumerate(tIJAB):
#         if not np.isclose(value, 0.0, atol=threshold):
#             variables[(2 * (nocc + A), 2 * I, 2 * (nocc + B), J)] = value

# print(len(variables))
# variables

In [ ]:
generator

In [ ]:
mp2_ansatz = expm(get_sparse_operator(0.5*excite_op, n_qubits=H_ferm.n_qubits))
## mp2_ansatz = expm(get_sparse_operator(excite_op, n_qubits=H_ferm.n_qubits))

## help(qubit_operator_sparse)

In [ ]:
mp2_ket =  mp2_ansatz @ hf_ket
print(np.allclose(mp2_ket, hf_ket))
hf_ket.conj().T @ H_mat @ mp2_ket

In [ ]:
pyscf_obj.pyscf_mp2.e_tot

In [ ]:
# tIjAb=np.einsum('abij -> abij', amplitudes, optimize='greedy')

# tIjAb.shape

In [ ]:
# pyscf_obj.pyscf_mp2.t2 # <-- these are the amplitudes!

In [ ]:
# variables = {}
# if tIjAb is not None:
#     nvirt = tIjAb.shape[2]
#     nocc = tIjAb.shape[0]
#     assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
#     for (I, J, A, B), value in np.ndenumerate(tIjAb):
#         if not np.isclose(value, 0.0, atol=1e-8):
#             variables[(nocc + A, I, nocc + B, J)] = value
            

nvirt = tIjAb.shape[2]
nocc = tIjAb.shape[0]
excite_op =FermionOperator()
assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
for (I, J, A, B), value in np.ndenumerate(tIjAb):
    if not np.isclose(value, 0.0, atol=1e-8):
        excite_op+= FermionOperator(f'{nocc + A}^ {I}^ {nocc + B} {J}', value)
#         excite_op+= FermionOperator(f'{I}^ {J}^ {nocc + A} {nocc + B}', value)
#         excite_op+= FermionOperator(f'{nocc + A}^ {nocc + B}^ {I} {J}', -value)

In [ ]:
tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

nvirt = tijab.shape[2]
nocc = tijab.shape[0]
excite_op =FermionOperator()
assert (tijab.shape[1] == nocc and tijab.shape[3] == nvirt)
for (I, J, A, B), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=1e-8):
        excite_op+= FermionOperator(f'{nocc + A}^ {I}^ {nocc + B} {J}', value)

In [ ]:
tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

tIjAb=tIjAb
tiJaB=tIjAb
tijab=tijab
tIJAB=tijab


threshold = 1e-8
excite_op =FermionOperator()
nvirt = tIjAb.shape[2]
nocc = tIjAb.shape[0]
assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)

for (I, j, A, b), value in np.ndenumerate(tIjAb):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + A)}^ {2*I}^ {2 * (nocc + b) + 1} {j + 1}', value/2)
#         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + b) + 1, j + 1)] = value
for (i, J, a, B), value in np.ndenumerate(tiJaB):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + a)+1}^ {2*i+1}^ {2 * (nocc + B)} {J}', value/2)
#         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + B), J)] = value
for (i, j, a, b), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + a)+1}^ {2*i+1}^ {2 * (nocc + b)+1} {j+1}', value/2)
#         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + b) + 1, j + 1)] = value
for (I, J, A, B), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=threshold):
#         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + B), J)] = value
        excite_op+= FermionOperator(f'{2 * (nocc + A)}^ {2*I}^ {2 * (nocc + B)} {J}', value/2)

In [ ]:
# tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

# tIjAb=tIjAb
# tiJaB=tIjAb
# tijab=tijab
# tIJAB=tijab


# threshold = 1e-8
# excite_op =FermionOperator()
# nvirt = tIjAb.shape[2]
# nocc = tIjAb.shape[0]
# assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)

# for (I, j, A, b), value in np.ndenumerate(tIjAb):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + A), 2*I, 2 * (nocc + b) + 1, j + 1]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + b) + 1, j + 1)] = value
# for (i, J, a, B), value in np.ndenumerate(tiJaB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + a)+1, 2*i+1, 2 * (nocc + B), J]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + B), J)] = value
# for (i, j, a, b), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + a)+1, 2*i+1, 2 * (nocc + b)+1, j+1]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + b) + 1, j + 1)] = value
# for (I, J, A, B), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + A), 2*I, 2 * (nocc + B), J]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)

In [ ]:
len(list(excite_op))

# excite_op/2

In [ ]:
len(list(generator))

In [ ]:
generator

In [ ]:
mp2_ansatz = expm(get_sparse_operator(0.25*excite_op, n_qubits=H_ferm.n_qubits))
## mp2_ansatz = expm(get_sparse_operator(excite_op, n_qubits=H_ferm.n_qubits))

## help(qubit_operator_sparse)

In [ ]:
mp2_ket =  mp2_ansatz @ hf_ket
mp2_ket.conj().T @ H_mat @ mp2_ket

In [ ]:
pyscf_obj.pyscf_mp2.e_tot

In [ ]:
pyscf_obj.pyscf_mp2.e_hf

In [ ]:
hf_ket.conj().T @ mp2_ket

In [ ]:
# t(a,i,b,j) = 0.25 * g(a,i,b,j)/(e(i) + e(j) -a(i) - b(j) )

# +aibj
# -abij

In [ ]:
pyscf_obj.pyscf_mp2.e_corr #+ pyscf_obj.pyscf_mp2.e_corr

In [ ]:
variables = {}
if tIjAb is not None:
    nvirt = tIjAb.shape[2]
    nocc = tIjAb.shape[0]
    assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
    for (I, J, A, B), value in np.ndenumerate(tIjAb):
        if not np.isclose(value, 0.0, atol=1e-8):
            variables[(nocc + A, I, nocc + B, J)] = value

dict(sorted(variables.items(), key=lambda x: np.abs(x[1]), reverse=True))

In [ ]:
from pyscf.cc.addons import spatial2spin

t2 = spatial2spin(pyscf_obj.pyscf_mp2.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

In [ ]:
# double_amplitudes_list=[]
# double_amplitudes = double_amps
# for i, j, k, l in zip(*double_amplitudes.nonzero()):
#     double_amplitudes_list.append([[i, j, k, l],
#                                    double_amplitudes[i, j, k, l]])
    
# double_amplitudes_list

double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])

print(len(double_amplitudes_list))
double_amplitudes_list

In [ ]:
generator = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [ ]:
generator

In [ ]:
mp2_ansatz = expm(get_sparse_operator(0.25*generator, n_qubits=H_ferm.n_qubits))
# mp2_ansatz = expm(get_sparse_operator(0.125*generator, n_qubits=H_ferm.n_qubits))

In [ ]:
mp2_ket =  mp2_ansatz @ hf_ket
mp2_ket.conj().T @ H_mat @ mp2_ket

In [ ]:
pyscf_obj.pyscf_mp2.e_tot

In [ ]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
ei = fij[:nocc]
ai = fij[nocc:]
V = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (
        ei.reshape(1, 1, -1, 1) + ei.reshape(1, 1, 1, -1) - ai.reshape(-1, 1, 1, 1) - ai.reshape(1, -1, 1, 1))
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.154.2423&rep=rep1&type=pdf

In [ ]:
mp2_ansatz = expm(get_sparse_operator(0.5*generator, n_qubits=H_ferm.n_qubits))
# mp2_ansatz = expm(get_sparse_operator(0.125*generator, n_qubits=H_ferm.n_qubits))

In [ ]:
mp2_ket =  mp2_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ mp2_ket

In [ ]:
pyscf_obj.pyscf_mp2.e_corr

In [ ]:
(hf_ket.conj().T @ H_mat @ mp2_ket - pyscf_obj.pyscf_mp2.e_hf).real

In [ ]:
H_new = H_mat @ expm(get_sparse_operator(0.5*generator, n_qubits=H_ferm.n_qubits))

(hf_ket.conj().T @ H_new @ hf_ket)